In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from shapely import wkt

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [34]:
pop_data = pd.DataFrame()

In [35]:
dens_data = pd.DataFrame()

In [1]:
city = 'Aguascalientes'

In [4]:
# Load proximity hexres9 for city
prox_schema = 'prox_analysis'
prox_table = 'proximityanalysis_hexres9'
query = f"SELECT * FROM {prox_schema}.{prox_table} WHERE \"city\" LIKE \'{city}\'"
prox_city = aup.gdf_from_query(query, geometry_col='geometry')

In [5]:
#--------------- PROCESS DATA ---------------
# Create time categories
prox_city['prox_cat'] = np.nan
prox_city.loc[prox_city.max_time>60 , 'prox_cat'] = '+60'
prox_city.loc[(prox_city.max_time>45 )&
             (prox_city.max_time<=60), 'prox_cat'] = '45a60'
prox_city.loc[(prox_city.max_time>30)&
             (prox_city.max_time<=45), 'prox_cat'] = '30a45'
prox_city.loc[(prox_city.max_time>15)&
             (prox_city.max_time<=30), 'prox_cat'] = '15a30'
prox_city.loc[(prox_city.max_time<=15), 'prox_cat'] = '0a15'

categories = ['0a15', '15a30', '30a45', '45a60', '+60']
prox_city['prox_cat'] = pd.Categorical(prox_city['prox_cat'], categories=categories, ordered=True)

# Show
print(prox_city.shape)
prox_city.head(2)

(1459, 42)


,hex_id_9,max_escuelas,max_preescolar,max_primaria,max_secundaria,max_servicios comunitarios,max_salud,max_guarderías,max_asistencia social,max_comercio,...,idx_cultural,mean_time,median_time,max_time,idx_sum,pobtot,dens_pobha,city,geometry,prox_cat
0,89498e3415bffff,44.295673,44.295673,44.164955,39.909090,70.033101,38.343919,52.648734,70.033101,42.827485,...,7.929043e-07,49.792875,41.368288,125.899925,2.943354,1279.0,103.649357,Aguascalientes,"POLYGON ((-102.24951 21.95406, -102.24827 21.9...",+60
1,89498eacaa3ffff,30.169718,23.001062,27.591513,30.169718,31.683427,31.683427,29.980341,3.093656,22.109336,...,1.994677e-01,19.818388,22.555199,39.486692,10.086212,0.0,0.000000,Aguascalientes,"POLYGON ((-102.28485 21.83514, -102.28361 21.8...",30a45


In [31]:
# Group by data
pobtot_summary = prox_city.groupby('prox_cat').agg({'pobtot':np.sum})
pobtot_summary.rename(columns={'pobtot':city},inplace=True)
pobtot_transposed = pobtot_summary.transpose()

# Show
pobtot_transposed

prox_cat,0a15,15a30,30a45,45a60,+60
Aguascalientes,36458.0,192168.0,182111.0,155767.0,476669.0


In [30]:
# Group by data
dens_summary = prox_city.groupby('prox_cat').agg({'dens_pobha':np.mean})
dens_summary.rename(columns={'dens_pobha':city},inplace=True)
dens_transposed = dens_summary.transpose()

# Show
dens_transposed

prox_cat,0a15,15a30,30a45,45a60,+60
Aguascalientes,52.754653,64.883608,64.445442,58.171618,53.873957


In [36]:
pop_data = pd.concat([pop_data,pobtot_transposed])
pop_data

prox_cat,0a15,15a30,30a45,45a60,+60
Aguascalientes,36458.0,192168.0,182111.0,155767.0,476669.0


In [37]:
dens_data = pd.concat([dens_data,dens_transposed])
dens_data

prox_cat,0a15,15a30,30a45,45a60,+60
Aguascalientes,52.754653,64.883608,64.445442,58.171618,53.873957


In [2]:
#Load mun data
mun_schema = 'metropolis'
mun_table = 'metro_gdf'
query = f"SELECT * FROM {mun_schema}.{mun_table}" 
gdf_mun = aup.gdf_from_query(query, geometry_col='geometry')

#Run main process
pop_data = pd.DataFrame()
dens_data = pd.DataFrame()

for city in gdf_mun.city.unique():
    
    print(f'STARTING ANALYSIS FOR {city}')
    
    #--------------- DOWNLOAD DATA ---------------
    # Load proximity hexres9 for city
    prox_schema = 'prox_analysis'
    prox_table = 'proximityanalysis_hexres9'
    query = f"SELECT * FROM {prox_schema}.{prox_table} WHERE \"city\" LIKE \'{city}\'"
    prox_city = aup.gdf_from_query(query, geometry_col='geometry')
    
    #--------------- PROCESS DATA ---------------
    # Create time categories
    prox_city['prox_cat'] = np.nan
    prox_city.loc[prox_city.max_time>60 , 'prox_cat'] = '+60'
    prox_city.loc[(prox_city.max_time>45 )&
                 (prox_city.max_time<=60), 'prox_cat'] = '45a60'
    prox_city.loc[(prox_city.max_time>30)&
                 (prox_city.max_time<=45), 'prox_cat'] = '30a45'
    prox_city.loc[(prox_city.max_time>15)&
                 (prox_city.max_time<=30), 'prox_cat'] = '15a30'
    prox_city.loc[(prox_city.max_time<=15), 'prox_cat'] = '0a15'

    categories = ['0a15', '15a30', '30a45', '45a60', '+60']
    prox_city['prox_cat'] = pd.Categorical(prox_city['prox_cat'], categories=categories, ordered=True)

    # Group by pop data
    pobtot_summary = prox_city.groupby('prox_cat').agg({'pobtot':np.sum})
    pobtot_summary.rename(columns={'pobtot':city},inplace=True)
    pobtot_transposed = pobtot_summary.transpose()
    pop_data = pd.concat([pop_data,pobtot_transposed])
    
    # Group by dens data
    dens_summary = prox_city.groupby('prox_cat').agg({'dens_pobha':np.mean})
    dens_summary.rename(columns={'dens_pobha':city},inplace=True)
    dens_transposed = dens_summary.transpose()
    dens_data = pd.concat([dens_data,dens_transposed])
    
    print(f'ANALIZED DATA FOR {city}')

STARTING ANALYSIS FOR Aguascalientes


OperationalError: (psycopg2.OperationalError) connection to server at "hippocampus.cswst4rid7eb.us-east-2.rds.amazonaws.com" (3.15.116.108), port 5432 failed: SSL SYSCALL error: Success
connection to server at "hippocampus.cswst4rid7eb.us-east-2.rds.amazonaws.com" (3.15.116.108), port 5432 failed: FATAL:  remaining connection slots are reserved for non-replication superuser connections

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [40]:
pop_data

prox_cat,0a15,15a30,30a45,45a60,+60
Aguascalientes,36458.0,192168.0,182111.0,155767.0,476669.0
Monclova,9148.0,84664.0,77786.0,68677.0,126049.0


In [41]:
dens_data

prox_cat,0a15,15a30,30a45,45a60,+60
Aguascalientes,52.754653,64.883608,64.445442,58.171618,53.873957
Monclova,28.501748,45.118971,37.731184,36.362263,24.719014
